In [ ]:
#SALIENCY MAPS

In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import os.path
import time
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
sns.set_style('darkgrid')
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from IPython.display import Image, display
import matplotlib.cm as cm
import PIL
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import warnings
# Adjusting the maximum number of columns for the DataFrame
pd.options.display.max_columns = None
pd.options.display.max_rows = 90
warnings.simplefilter("ignore")

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def input_img(path):
    image = tf.image.decode_image(tf.io.read_file(path))
    image = preprocess_image(image)
    return image

def preprocess_image(image):
    # Resize image to 224x224
    image = tf.image.resize(image, [224, 224])
    # Ensure the image is in RGB format
    if image.shape[-1] != 3:
        image = tf.image.grayscale_to_rgb(image)
    return image

def normalize_image(img):
    grads_norm = img[:,:,0]+ img[:,:,1]+ img[:,:,2]
    grads_norm = (grads_norm - tf.reduce_min(grads_norm))/ (tf.reduce_max(grads_norm)- tf.reduce_min(grads_norm))
    return grads_norm

def plot_maps(img1, img2, vmin=0.3, vmax=0.7, mix_val=2):
    f = plt.figure(figsize=(15,45))
    plt.subplot(1,3,1)
    plt.imshow(img1, vmin=vmin, vmax=vmax, cmap="ocean")
    plt.axis("off")
    plt.title('Saliency Map')
    plt.subplot(1,3,2)
    plt.imshow(img2, cmap="ocean")
    plt.axis("off")
    plt.title('Input Image')
    plt.subplot(1,3,3)
    plt.imshow(img1*mix_val+img2/mix_val, cmap="ocean" )
    plt.axis("off")
    plt.title('Overlayed Image')

# Load your trained model
model_path =  # Update with model path
test_model = tf.keras.models.load_model(model_path)

# Define the path to the image on your Google Drive
img_path =  # Update with test image

# Read and preprocess the image
input_img = input_img(img_path)
input_img = tf.expand_dims(input_img, axis=0)

# Make predictions using your model
result = test_model.predict(input_img)
max_idx = np.argmax(result,axis = 1)
print("Predicted class indices:", max_idx)

# Generate saliency map
with tf.GradientTape() as tape:
    tape.watch(input_img)
    result = test_model(input_img)
    max_score = result[0,max_idx[0]]

grads = tape.gradient(max_score, input_img)
plot_maps(normalize_image(grads[0]), normalize_image(input_img[0]))


In [ ]:
#GRADCAM

In [ ]:
test_model.summary()

In [ ]:
from IPython.display import Image, display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras

img_size = (224, 224)

def preprocess_image(image):
    # Resize image to 224x224
    image = tf.image.resize(image, [224, 224])
    # Ensure the image is in RGB format
    if image.shape[-1] != 3:
        image = tf.image.grayscale_to_rgb(image)
    return image

def normalize_image(img):
    grads_norm = img[:,:,0]+ img[:,:,1]+ img[:,:,2]
    grads_norm = (grads_norm - tf.reduce_min(grads_norm))/ (tf.reduce_max(grads_norm)- tf.reduce_min(grads_norm))
    return grads_norm

last_conv_layer_name = "Conv_1"

## The local path to our target image

input_img1 = # Update with test image

display(Image(input_img1))

def get_img_array(img_path, size):
    ## `img` is a PIL image
    img = keras.utils.load_img(img_path, target_size=size)
    array = keras.utils.img_to_array(img)
    ## We add a dimension to transform our array into a "batch"
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    ## First, we create a model that maps the input image to the activations
    ## of the last conv layer as well as the output predictions
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )

    ## Then, we compute the gradient of the top predicted class for our input image
    ## for the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    ## We are doing transfer learning on last layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    ## This is a vector where each entry is the mean intensity of the gradient
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    ## calculates a heatmap highlighting the regions of importance in an image
    ## for a specific
    ## predicted class by combining the output of the last convolutional layer
    ## with the pooled gradients.
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    ## For visualization purpose
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
## Preparing the image
img_array = preprocess_image(get_img_array(input_img1, size=img_size))

preds = test_model.predict(img_array)
print("Predicted of image:", preds)

## Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, test_model, last_conv_layer_name)

## visulization of heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="save_cam_image.jpg", alpha=0.4):
    ## Loading the original image
    img = keras.utils.load_img(img_path)
    img = keras.utils.img_to_array(img)

    ## Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    ## Use jet colormap to colorize heatmap
    jet = mpl.colormaps["jet"]

    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    ## Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    ## Superimpose the heatmap on original image
    Superimposed_img = jet_heatmap * alpha + img
    Superimposed_img = keras.utils.array_to_img(Superimposed_img)

    ## Save the superimposed image
    Superimposed_img.save(cam_path)

    ## Displaying Grad CAM
    display(Image(cam_path))


save_and_display_gradcam(input_img1, heatmap)